# GSI Land Classification Primer
#### Author: Alexey Tarutin

This notebook gives an overview of the proposed GSI land classification mechanism, alongside results from a previous project in the same area of interest. The point of this is to improve upon this using notions of textural and windowed predictions based purely on optical imagery from a single date.

This notebook outlines the following:
* Reading the data
* Clipping the data to the AOI
* Preparing the data for machine learning
* Training/prediction
* Accuracy validation
* Post-processing
* Writing output

Supplemental material to this notebook:
* In the data/ directory
 * A Level 2 Sentinel-2 Scene from 28th of November 2017
 * A land classification map based on Corine from 2018 for training
 * Field boundaries for the area courtesy of the Centre for Ecology and Hydrology for aggregation of results
 * Validation boundaries created by GSI
* environment.yml to create the Anaconda environment on your local machine. If you don't use Anaconda, please install the Python 3 distribution for your system: https://docs.conda.io/en/latest/miniconda.html
 * This is populated with the following command: `conda env create -f environment.yml`
 * Activate with: `conda activate geo`

In [ ]:
import rasterio as rio
import geopandas as gpd
import os
import sys
from glob import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import pylab as pl
from fiona.crs import from_epsg
from shapely.geometry import box as geobox
from rasterstats import zonal_stats

## Step 1
Create a GeoJSON file using the coordinates for the area. This is to allow us to validate and verify data fit.

In [2]:
aoi_geo = geobox(-2.29, 51.51, -1.71, 51.61)
aoi = gpd.GeoDataFrame([], geometry=[aoi_geo])
aoi.crs = from_epsg(4326)
aoi.to_file('./data/aoi.geojson', driver='GeoJSON')

We have prepared a package named `land_classification` that gives a simple structure for file IO, dataset sampling and cleaning, machine learning, output validation, and postprocessing.

You are welcome to modify functions in this as appropriate. Expect bugs.

In [3]:
import land_classification as lc

## Step 2
Load in Sentinel-2 scene data. 

Notes:
* We are only using a single scene for this exercise. We want to maximise our ability to create land classification maps on only one image, this will also help us extend this to other applications such as change detection and phenology analysis.
* This scene was downloaded from the Sentinel-2 Google Cloud Storage repository in SAFE format as Level-1C. This file was then processed to Level-2A using Sen2Cor for the advantages it provides, including atmospheric correction and improved cloud detection. Further information about level 2 Sentinel-2 imagery, advantages, and the relevant algorithms is available here: https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm

In [4]:
s2_band = 'S2A_MSIL2A_20171128T111411_N0206_R137_T30UWC_20171128T130743.SAFE'
data, profile = lc.merge_bands(s2_band)
lc.write_raster('data/merged.tif', data, profile)

Writing raster data/merged.tif with 4 bands.


CRSError: The WKT could not be parsed. OGR Error code 7

In [5]:
lc.mask_raster(aoi, 'data/merged.tif', 'data/masked.tif')

Writing raster data/masked.tif with 4 bands.
Writing band 1 of 4
Writing band 2 of 4
Writing band 3 of 4
Writing band 4 of 4
File data/masked.tif written.


In [4]:
lc.copy_projection('data/masked.tif', 'data/Corine_Orig_10m_OS_AoI1.tif','data/Corine_S2_Proj_2.tif')

In [4]:
mask_src = rio.open('data/masked.tif')
profile = mask_src.profile
c_tif = rio.open('data/Corine_S2_Proj_2.tif')
c_data = c_tif.read()
data = mask_src.read(list(pl.arange(mask_src.count) + 1))
gdf = lc.create_raster_df(data)
gdf = lc.calc_indices(gdf)
gdf['labels'] = lc.create_raster_df(pred_array=c_data, bands=['labels'])['labels']

In [5]:
clean_df = lc.remove_outliers(gdf)

clean_df = lc.create_zero_samples(clean_df)
clean_df = lc.calc_indices(clean_df)
clean_df = lc.onehot_targets(clean_df)

/home/alexey/GSI/projects/msc/land_classification/land_classification/preprocessing.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  lab_clean['labels'] = i


The original projection of this was incorrect. The file you receive should have the same projection as the S2 data. If it doesn't, run the below line.

In [ ]:
pred, proba, cm, cls = lc.classify(clean_df, name='cv_mlp', bands=None)

# Window based clustering

Below is a naive implementation of aggregating neighbouring pixels by a window. This is more of a illustrative attempt and is very much subject to improvement. These include
* Using a variable stride for the sliding window. Right now it's a constant based on the window size
* Random sampling of image
* Allow prediction on class patches

In [103]:
win_res = 10 ## Window resolution of 10 implies 100m pixels to predict
data_30 = pl.zeros(shape=(data.shape[0], int(data.shape[1]/win_res), int(data.shape[1]/win_res), win_res, win_res))

feature_stack = pl.zeros(shape=(pl.array(data_30.shape[1:3]).prod(), data_30.shape[0]*pl.array(data_30.shape[3:]).prod()))
k = 0
for i in pl.arange(data_30.shape[1]):
    for j in pl.arange(data_30.shape[2]):
        tmp_ar = data[:, (i*win_res):(i*win_res)+win_res, (j*win_res):(j*win_res)+win_res]
        row = pl.zeros((win_res ** 2) * data.shape[0])
        for l, a in enumerate(tmp_ar):
            af = a.flatten()
            pl.shuffle(af)
            row[(l*(win_res**2)):(l*(win_res**2)) + (win_res**2)] = af
        feature_stack[k, :] = row
        k += 1
feature_stack = pl.stack(feature_stack)